In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import torch


In [2]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from matplotlib.pyplot import imshow
from PIL import Image
import os
import numpy as np
import json
from detectron2.structures import BoxMode
def cv2_imshow(img):
    img = img[:,:,[2,1,0]]
    img = Image.fromarray(img)
    display(img)
    '''plt.figure(figsize=(20, 20))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    #img = transforms.ToPILImage(img)
    #plt.show(img)
    '''


In [3]:
people_jsonlist = []
people_imagelist = []
with open('/root/notebooks/Module_final/pedestrian_generator/Detectron2/people_jsonlist_final_v2', 'r') as j:
    lines = j.readlines()
    for line in lines:
        people_jsonlist.append(line[:-1])
with open('/root/notebooks/Module_final/pedestrian_generator/Detectron2/people_imagelist_final_v2', 'r') as i:
    lines = i.readlines()
    for line in lines:
        people_imagelist.append(line[:-1])

print(len(people_jsonlist), len(people_imagelist))
print(people_imagelist[:2])
print(people_jsonlist[:2])

46024 46024
['/root/notebooks/Module_final/caltech_origin_data_refine/street/045034.jpg', '/root/notebooks/Module_final/caltech_origin_data_refine/street/031156.jpg']
['/root/notebooks/Module_final/caltech_origin_data_refine/street_json/045034.json', '/root/notebooks/Module_final/caltech_origin_data_refine/street_json/031156.json']


In [4]:
train_people_jsonlist = people_jsonlist[:-4024]
train_people_imagelist = people_imagelist[:-4024]
test_people_jsonlist = people_jsonlist[-4024:]
test_people_imagelist = people_imagelist[-4024:]
print(len(train_people_imagelist))
print(len(train_people_jsonlist))

print(len(test_people_imagelist))
print(len(test_people_jsonlist))

print(test_people_jsonlist[0])

42000
42000
4024
4024
/root/notebooks/Module_final/caltech_origin_data_refine/street_json/096474.json


In [5]:
def get_pedestrain_dict(image_list, json_list):
    dataset_dicts = []
    
    for i,path in tqdm(enumerate(image_list)):
        filename = path
        #img = cv2.imread(path)
        # height, width = cv2.imread(filename).shape[:2]
        record = {}
        record['file_name'] = filename
        record['image_id'] = i #path.split('/')[-1][:-5]
        #id is like 000000 or 000001
        #record['file_img'] = img
        record['height']= 480
        record['width']= 640
        
        #for i in data_list[1] to get bbox and category
        objs = []
        
        people = json_list[i]
        with open(people) as p:
            json_context = json.load(p)
            for person in json_context:
                boxes = list(map(float, person['pos']))
                obj = {
                    "bbox": boxes,
                    "bbox_mode": BoxMode.XYWH_ABS,
                    #"segmentation": [poly], To draw a line, along to ballon
                    "category_id": 0,
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts #list of dicts

In [6]:
from detectron2.data import DatasetCatalog, MetadataCatalog

DatasetCatalog.register("pedestrain_test", lambda i=i: get_pedestrain_dict(test_people_imagelist, test_people_jsonlist))
MetadataCatalog.get("pedestrain_test").set(thing_classes=["person"])

Metadata(name='pedestrain_test', thing_classes=['person'])

In [18]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Base-RCNN-FPN.yaml"))
#cfg.DATASETS.TRAIN = ("pedestrain_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"#model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
#cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER =  156000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.OUTPUT_DIR = '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/'

In [30]:
import glob
model_list = sorted(glob.glob('/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model*'))
print(model_list)
print(len(model_list))

['/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0004999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0009999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0014999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0019999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0024999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0029999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0034999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0039999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0044999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0049999.pth', '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0054999.pth', '/root/no

In [32]:
model_list

['/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0004999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0009999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0014999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0019999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0024999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0029999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0034999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0039999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0044999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0049999.pth',
 '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_0054999.pth',

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader
evaluation_list = []
for model in model_list:
    cfg.MODEL.WEIGHTS = model
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
    cfg.DATASETS.TEST = ("pedestrain_test", )
    
    predictor = DefaultPredictor(cfg)
    COCOEvaluator.reset
    evaluator = COCOEvaluator("pedestrain_test", cfg, False, output_dir = cfg.OUTPUT_DIR)

    val_loader = build_detection_test_loader(cfg, "pedestrain_test")
    evaluation_list.append(inference_on_dataset(predictor.model, val_loader, evaluator))

4024it [00:06, 647.14it/s]


[06/26 23:16:01 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/26 23:16:01 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/26 23:16:01 d2.evaluation.evaluator]: Start inference on 4024 images
[06/26 23:16:02 d2.evaluation.evaluator]: Inference done 11/4024. 0.0395 s / img. ETA=0:02:42
[06/26 23:16:07 d2.evaluation.evaluator]: Inference done 134/4024. 0.0393 s / img. ETA=0:02:38
[06/26 23:16:12 d2.evaluation.evaluator]: Inference done 258/4024. 0.0392 s / img. ETA=0:02:32
[06/26 23:16:17 d2.evaluation.evaluator]: Inference done 382/4024. 0.0392 s / img. ETA=0:02:27
[06/26 23:16:22 d2.evaluation.evaluator]: Inference done 507/4024. 0.0391 s / img. ETA=0:02:22
[06/26 23:16:27 d2.evaluation.evaluator]: Inference done 632/4024. 0.0391 s / img. ETA=0:02:17
[06/26 23:16:32 d2.evaluation.evaluator]: Inference done 756/4024. 0.0390 s / img. ETA=0:02:11
[06/26 23:16:37 d2.evaluation.evaluator]: Inference done 880/4024. 0.0391 s / img. ETA=0:0

4024it [00:06, 646.42it/s]


[06/26 23:18:54 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/26 23:18:54 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/26 23:18:54 d2.evaluation.evaluator]: Start inference on 4024 images
[06/26 23:18:55 d2.evaluation.evaluator]: Inference done 11/4024. 0.0390 s / img. ETA=0:02:40
[06/26 23:19:00 d2.evaluation.evaluator]: Inference done 136/4024. 0.0389 s / img. ETA=0:02:36
[06/26 23:19:05 d2.evaluation.evaluator]: Inference done 261/4024. 0.0389 s / img. ETA=0:02:31
[06/26 23:19:10 d2.evaluation.evaluator]: Inference done 385/4024. 0.0390 s / img. ETA=0:02:26
[06/26 23:19:15 d2.evaluation.evaluator]: Inference done 509/4024. 0.0390 s / img. ETA=0:02:21
[06/26 23:19:20 d2.evaluation.evaluator]: Inference done 632/4024. 0.0391 s / img. ETA=0:02:16
[06/26 23:19:25 d2.evaluation.evaluator]: Inference done 757/4024. 0.0390 s / img. ETA=0:02:11
[06/26 23:19:30 d2.evaluation.evaluator]: Inference done 880/4024. 0.0391 s / img. ETA=0:0

4024it [00:07, 543.07it/s]


...

[06/26 23:57:24 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/26 23:57:24 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/26 23:57:24 d2.evaluation.evaluator]: Start inference on 4024 images
[06/26 23:57:25 d2.evaluation.evaluator]: Inference done 11/4024. 0.1083 s / img. ETA=0:07:19
[06/26 23:57:30 d2.evaluation.evaluator]: Inference done 61/4024. 0.1017 s / img. ETA=0:06:49
[06/26 23:57:35 d2.evaluation.evaluator]: Inference done 113/4024. 0.0996 s / img. ETA=0:06:35
[06/26 23:57:40 d2.evaluation.evaluator]: Inference done 163/4024. 0.0996 s / img. ETA=0:06:30
[06/26 23:57:45 d2.evaluation.evaluator]: Inference done 215/4024. 0.0986 s / img. ETA=0:06:21
[06/26 23:57:50 d2.evaluation.evaluator]: Inference done 265/4024. 0.0987 s / img. ETA=0:06:16
[06/26 23:57:55 d2.evaluation.evaluator]: Inference done 318/4024. 0.0981 s / img. ETA=0:06:08
[06/26 23:58:01 d2.evaluation.evaluator]: Inference done 368/4024. 0.0986 s / img. ETA=0:06

4024it [00:06, 629.08it/s]


[06/27 00:04:15 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 00:04:15 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 00:04:15 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 00:04:16 d2.evaluation.evaluator]: Inference done 11/4024. 0.1147 s / img. ETA=0:07:45
[06/27 00:04:21 d2.evaluation.evaluator]: Inference done 61/4024. 0.1008 s / img. ETA=0:06:45
[06/27 00:04:26 d2.evaluation.evaluator]: Inference done 111/4024. 0.1000 s / img. ETA=0:06:37
[06/27 00:04:31 d2.evaluation.evaluator]: Inference done 162/4024. 0.0996 s / img. ETA=0:06:30
[06/27 00:04:36 d2.evaluation.evaluator]: Inference done 214/4024. 0.0990 s / img. ETA=0:06:22
[06/27 00:04:42 d2.evaluation.evaluator]: Inference done 265/4024. 0.0988 s / img. ETA=0:06:16
[06/27 00:04:47 d2.evaluation.evaluator]: Inference done 315/4024. 0.0988 s / img. ETA=0:06:11
[06/27 00:04:52 d2.evaluation.evaluator]: Inference done 367/4024. 0.0984 s / img. ETA=0:06

4024it [00:06, 648.46it/s]


[06/27 00:11:09 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/27 00:11:09 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/27 00:11:09 d2.evaluation.evaluator]: Start inference on 4024 images
[06/27 00:11:11 d2.evaluation.evaluator]: Inference done 11/4024. 0.1082 s / img. ETA=0:07:18
[06/27 00:11:16 d2.evaluation.evaluator]: Inference done 62/4024. 0.0985 s / img. ETA=0:06:35
[06/27 00:11:21 d2.evaluation.evaluator]: Inference done 114/4024. 0.0968 s / img. ETA=0:06:24
[06/27 00:11:26 d2.evaluation.evaluator]: Inference done 164/4024. 0.0975 s / img. ETA=0:06:21
[06/27 00:11:31 d2.evaluation.evaluator]: Inference done 214/4024. 0.0982 s / img. ETA=0:06:19
[06/27 00:11:36 d2.evaluation.evaluator]: Inference done 266/4024. 0.0978 s / img. ETA=0:06:12
[06/27 00:11:41 d2.evaluation.evaluator]: Inference done 316/4024. 0.0983 s / img. ETA=0:06:09
[06/27 00:11:46 d2.evaluation.evaluator]: Inference done 368/4024. 0.0980 s / img. ETA=0:06

In [22]:
print(evaluation_list)

[OrderedDict([('bbox', {'AP': nan, 'AP50': nan, 'AP75': nan, 'APs': nan, 'APm': nan, 'APl': nan})]), OrderedDict([('bbox', {'AP': 0.7920792079207918, 'AP50': 0.99009900990099, 'AP75': 0.99009900990099, 'APs': 0.7920792079207921, 'APm': 0.34653465346534656, 'APl': 0.0})]), OrderedDict([('bbox', {'AP': 0.8779719454349669, 'AP50': 1.6733252272595676, 'AP75': 0.99009900990099, 'APs': 0.8221499860829456, 'APm': 0.5514302970605705, 'APl': 1.7633191890617632})]), OrderedDict([('bbox', {'AP': 1.416113126198241, 'AP50': 4.146725692836759, 'AP75': 0.6430614033827186, 'APs': 1.2114060485487679, 'APm': 1.7479566807726332, 'APl': 1.782178217821782})]), OrderedDict([('bbox', {'AP': 1.474629156784471, 'AP50': 3.2071177114215255, 'AP75': 1.0274241709885275, 'APs': 1.5072754636384733, 'APm': 1.448637476076259, 'APl': 3.4396436074407877})]), OrderedDict([('bbox', {'AP': 2.7799631686706405, 'AP50': 8.201902180650603, 'AP75': 0.7229870296220217, 'APs': 2.4302054392025396, 'APm': 3.664571185313021, 'APl': 

In [44]:
iter_count = 0
baseline_x_AP, baseline_x_APs, baseline_x_APm, baseline_x_APl = [], [], [], []
baseline_y = []
for evaluation in evaluation_list[:-1]:
#     print(evaluation['bbox']['AP'])
    baseline_x_AP.append(evaluation['bbox']['AP'])
    baseline_x_APs.append(evaluation['bbox']['APs'])
    baseline_x_APm.append(evaluation['bbox']['APm'])
    baseline_x_APl.append(evaluation['bbox']['APl'])
    iter_count+=5000
    baseline_y.append(iter_count)
baseline_x_AP.append(evaluation_list[-1]['bbox']['AP'])
baseline_x_APs.append(evaluation_list[-1]['bbox']['APs'])
baseline_x_APm.append(evaluation_list[-1]['bbox']['APm'])
baseline_x_APl.append(evaluation_list[-1]['bbox']['APl'])
baseline_y.append(126000)
print(baseline_x_AP)
print(baseline_x_APs)
print(baseline_x_APm)
print(baseline_x_APl)
print(baseline_y)


[nan, 0.7920792079207918, 0.8779719454349669, 1.416113126198241, 1.474629156784471, 2.7799631686706405, 4.365055883599328, 5.072990736152868, 4.828097950619085, 6.92666897005893, 6.936715654165922, 9.44526238966597, 10.8450859504076, 11.620194694807868, 11.789532057145058, 12.419960184142699, 12.346690430765559, 12.070976885637704, 12.450554933323415, 12.46166899796417, 12.460662114153836, 12.430281003807856, 12.451646476443024, 12.47894003968933, 12.740091853089067]
[nan, 0.7920792079207921, 0.8221499860829456, 1.2114060485487679, 1.5072754636384733, 2.4302054392025396, 3.6748144383109644, 3.704140040844627, 3.724394555586799, 5.314299315507101, 4.99023635582738, 6.508431200874189, 7.994627176372333, 8.985816117106824, 8.825760648543813, 9.701303960662012, 9.296547184963565, 9.082830943728254, 9.50626196156194, 9.307251542205371, 9.442260123689392, 9.426795060677598, 9.403089944627991, 9.474870853038956, 9.666382180971317]
[nan, 0.34653465346534656, 0.5514302970605705, 1.7479566807726

In [24]:
len(evaluation_list)

25

In [17]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader
cfg.MODEL.WEIGHTS = '/root/notebooks/Module_final/detectron2_output/baseline_FPN_final_v2/model_final.pth'
predictor = DefaultPredictor(cfg)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("pedestrain_test", )
predictor = DefaultPredictor(cfg)
# COCOEvaluator.reset
evaluator = COCOEvaluator("pedestrain_test", cfg, False, output_dir = cfg.OUTPUT_DIR)

val_loader = build_detection_test_loader(cfg, "pedestrain_test")
inference_on_dataset(predictor.model, val_loader, evaluator)

4024it [00:06, 606.42it/s]


[06/26 23:12:35 d2.data.common]: Serializing 4024 elements to byte tensors and concatenating them all ...
[06/26 23:12:35 d2.data.common]: Serialized dataset takes 1.47 MiB
[06/26 23:12:35 d2.evaluation.evaluator]: Start inference on 4024 images
[06/26 23:12:36 d2.evaluation.evaluator]: Inference done 11/4024. 0.0402 s / img. ETA=0:02:46
[06/26 23:12:41 d2.evaluation.evaluator]: Inference done 134/4024. 0.0395 s / img. ETA=0:02:38
[06/26 23:12:46 d2.evaluation.evaluator]: Inference done 257/4024. 0.0394 s / img. ETA=0:02:33
[06/26 23:12:51 d2.evaluation.evaluator]: Inference done 380/4024. 0.0394 s / img. ETA=0:02:28
[06/26 23:12:56 d2.evaluation.evaluator]: Inference done 503/4024. 0.0394 s / img. ETA=0:02:23
[06/26 23:13:01 d2.evaluation.evaluator]: Inference done 626/4024. 0.0394 s / img. ETA=0:02:18
[06/26 23:13:06 d2.evaluation.evaluator]: Inference done 748/4024. 0.0394 s / img. ETA=0:02:13
[06/26 23:13:11 d2.evaluation.evaluator]: Inference done 871/4024. 0.0394 s / img. ETA=0:0

KeyboardInterrupt: 